In [ ]:
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *

# 矢量计算实现

In [ ]:
# 获取要素字段
def get_filed(ele_path):
    """获取输入要素的字段及类型"""
    field_dict = {}
    for one_field in arcpy.ListFields(ele_path):
        field_dict[one_field.name] = one_field.type
    return field_dict
# 获取要素的df表
def get_ele_df(ele_path,field_list):
    """获取输入要素的df表"""
    df = pd.DataFrame(TableToNumPyArray(ele_path,field_list))
    return df


# 按属性选择并导出为要素类
def export_ele_by_attr(in_path, out_path, field_name, field_values):
    """
    按属性选择并导出为要素类，field_values可以是单个值也可以是一个列表

    参数:
    in_path: 输入要素类的路径
    out_path: 输出要素类的路径
    field_name: 用于选择的字段名称
    field_values: 单个字段值或字段值列表，基于这些值进行选择
    """
    # 确保field_values总是列表
    if not isinstance(field_values, list):
        field_values = [field_values]  # 将单个值转换为列表

    # 将输入的要素类转换为图层
    temp_layer = "tempLayer"
    arcpy.MakeFeatureLayer_management(in_path, temp_layer)
    
    # 清除在临时图层上的任何现有选择
    arcpy.SelectLayerByAttribute_management(temp_layer, "CLEAR_SELECTION")
    
    # 构建查询语句以处理值列表
    values_str = ','.join([f"'{value}'" for value in field_values])
    query = f"{field_name} IN ({values_str})"
    
    # 使用查询语句按属性选择
    arcpy.SelectLayerByAttribute_management(temp_layer, "NEW_SELECTION", query)
    
    # 检查是否有选择的要素
    if int(arcpy.GetCount_management(temp_layer)[0]) > 0:
        # 导出选择的要素
        arcpy.CopyFeatures_management(temp_layer, out_path)
        print(f"已成功导出要素到 {out_path}")
    else:
        print("没有找到匹配的要素。")
    
    # 清理临时图层
    arcpy.Delete_management(temp_layer)




In [ ]:
work_path = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features_dy'
env.workspace = work_path
env.overwriteOutput = True

In [ ]:
arcpy.ListFeatureClasses()

In [ ]:
gtsd_path = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features_dy\dy_sd'
temp_path = r'D:\ArcGISProjects\workspace\shbyq\soil_type_result.gdb\soiltype_dy\soiltype_prediction_CATB_20240226'

In [ ]:
field_dict = get_filed(gtsd_path)

In [ ]:
field_dict

In [ ]:
df = get_ele_df(gtsd_path,['DLBM','DLMC'])
# 去重
df =df.drop_duplicates()
df

In [ ]:
df = get_ele_df(temp_path,['TL'])
# 去重
df =df.drop_duplicates()
df

In [ ]:
export_ele_by_attr(temp_path,'dy_pre_tl','TL','水稻土')

In [ ]:
export_ele_by_attr('dy_sd','dy_ZRT','DLMC',['茶园','其他园地','乔木林地','竹林地','灌木林地','其他林地','人工牧草地','其他草地'])

# 栅格计算实现

In [ ]:
import numpy as np
import arcpy
from scipy.spatial import cKDTree

In [ ]:
def load_raster_as_array(raster_path):
    """
    将栅格数据加载为NumPy数组。
    """
    return arcpy.RasterToNumPyArray(raster_path)

def create_cKDTree_for_replacement(raster, mask, inverse=False):
    """
    根据指定的掩码创建cKDTree，用于快速寻找最近的替代值。
    inverse参数决定是使用掩码(True)还是反掩码(False)中的位置创建树。
    """
    if inverse:
        positions = np.column_stack(np.where(~mask))
    else:
        positions = np.column_stack(np.where(mask))
    return cKDTree(positions), positions
def find_and_replace_with_mapping(pre_raster, dl_raster, mapping, paddy_codes):
    """
    根据dl_raster到pre_raster的映射关系，查找并替换需要调整的像素值。
    """
    corrected_raster = np.copy(pre_raster)
    for dl_code, target_pre_codes in mapping.items():
        # 找到dl_raster中指定dl_code的所有位置
        dl_mask = dl_raster == dl_code
        # 找出这些位置中pre_raster不在目标水田代码列表中的像素
        replace_mask = dl_mask & (~np.isin(pre_raster, target_pre_codes))
        # 对这些需要替换的像素进行处理
        corrected_raster = find_and_replace_nearest(corrected_raster, replace_mask, paddy_codes)
    
    return corrected_raster


def find_and_replace_nearest(target_raster, mask, replacement_codes, inverse=False):
    """
    查找并替换栅格中需要被替代的值。
    """
    corrected_raster = np.copy(target_raster)
    tree, valid_positions = create_cKDTree_for_replacement(target_raster, np.isin(target_raster, replacement_codes), inverse)
    to_be_replaced = np.column_stack(np.where(mask))
    
    if to_be_replaced.size > 0:
        _, nearest_indices = tree.query(to_be_replaced)
        nearest_valid_values = target_raster[valid_positions[nearest_indices][:, 0], valid_positions[nearest_indices][:, 1]]
        corrected_raster[to_be_replaced[:, 0], to_be_replaced[:, 1]] = nearest_valid_values

    return corrected_raster

In [ ]:
env.workspace = r'D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result\feature_soiltype_result_dy.gdb'

In [ ]:
# 示例使用
pre_raster_path = r'D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result\feature_soiltype_result_dy.gdb\prediction_sioltype_CATB_20240226'
dl_raster_path = r'D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\DL'
output_raster_path = "prediction_sioltype_CATB_20240226_reclass"

In [ ]:
# 定义类别代码
paddy_codes = [76, 11, 12, 25, 74, 79, 28, 17, 27, 80, 15, 21, 10, 20, 26, 22, 82]  # 水田代码
natural_soil_codes_dl = [5,6,7,8,9,10,11,12]  # dl_raster中自然土代码

# 条件1: dl_raster中标记为水田的像素在pre_raster中也必须是水田代码
# 映射字典
mapping = {
    1: paddy_codes,  # dl_raster中的1对应pre_raster中的所有水田代码
    2: paddy_codes,  # 同上
    31: paddy_codes,   # 同上
    32: paddy_codes   # 同上
}

In [ ]:
pre_raster = load_raster_as_array(pre_raster_path)
dl_raster= load_raster_as_array(dl_raster_path)

In [ ]:
pre_raster.shape, dl_raster.shape

In [ ]:
# 使用映射字典处理条件1
corrected_raster_condition1 = find_and_replace_with_mapping(pre_raster, dl_raster, mapping, paddy_codes)

# 条件2: pre_raster中为水田代码的像素，对应的dl_raster的值不应该在自然土代码列表中
natural_soil_mask_dl = np.isin(dl_raster, natural_soil_codes_dl)  # dl_raster中自然土的掩码
paddy_in_pre = np.isin(pre_raster, paddy_codes)  # pre_raster中水田的掩码
condition2_mask = natural_soil_mask_dl & paddy_in_pre
corrected_raster_condition2 = find_and_replace_nearest(corrected_raster_condition1, condition2_mask, paddy_codes, True)


In [ ]:
# 将结果转存为
corrected_raster_condition2

In [ ]:
env.extent = r'D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result\feature_soiltype_result_dy.gdb\prediction_sioltype_CATB_20240226'

In [ ]:
# 掩膜提取
def mask_raster(array,mask_ele,cell_size):
    out_raster = arcpy.NumPyArrayToRaster(
    array,
    arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
    cell_size,
    cell_size,
)
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    output_coordinate_system = arcpy.Describe(mask_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system,snapRaster=mask_ele, cellSize=mask_ele):
        result_raster = arcpy.sa.ExtractByMask(out_raster, mask_ele, "INSIDE")
        return result_raster

In [ ]:
result_raster = mask_raster(corrected_raster_condition2,r'D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result\feature_soiltype_result_dy.gdb\prediction_sioltype_CATB_20240226', 5)

In [ ]:
arcpy.sa.Int(result_raster)

In [ ]:
result_raster.save(r'D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result\feature_soiltype_result_dy.gdb\prediction_sioltype_CATB_20240226_reclass')